In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns

from scipy.stats import norm

# Chapter 1: Introduction to Data

## 1.1 Case Study: using stents to prevent strokes

In [2]:
# retrieve datasets
stent30 = pd.read_csv(
    'OI_Resources/stent30.csv',
    header=0,
    names=['group', '0_to_30_days']
)
stent365 = pd.read_csv(
    'OI_Resources/stent365.csv',
    header=0,
    names= ['group', '0_to_365_days']
)

# create stents dataset
stents = pd.concat(
    [stent30, stent365['0_to_365_days']],
    axis='columns'
)

# dataset rows and columns
print(stent30.shape, stent365.shape, stents.shape)
# sample 8 records
display(stents.sample(8))
# patient outcomes for treatment groups
# display(stents[['group', '0_to_30_days']].value_counts())
# display(stents[['group', '0_to_365_days']].value_counts())

(451, 2) (451, 2) (451, 3)


,group,0_to_30_days,0_to_365_days
225,control,stroke,stroke
251,control,no event,stroke
299,control,no event,no event
260,control,no event,no event
307,control,no event,no event
306,control,no event,no event
312,control,no event,no event
78,treatment,no event,no event


In [3]:
# patient outcomes for treatment groups
month_outcomes = pd.crosstab(stents['group'], stents['0_to_30_days'])
year_outcomes = pd.crosstab(stents['group'], stents['0_to_365_days'])
display(month_outcomes)
display(year_outcomes)

0_to_30_days,no event,stroke
group,,
control,214,13
treatment,191,33


0_to_365_days,no event,stroke
group,,
control,199,28
treatment,179,45


**Summary statistic**: A single number summarizing a large amount of data.

In [4]:
# proportion of patients in control group who had stroke within 1 year
year_control_stroke_prop = round((year_outcomes.loc['control', 'stroke']\
/ year_outcomes.loc['control'].sum())*100, 2)
# proportion of patients in treatment group who had stroke within 1 year
year_treatment_stroke_prop = round((year_outcomes.loc['treatment', 'stroke']\
/ year_outcomes.loc['treatment'].sum())*100, 2)

print('Proportion of strokes in 1-year control group:', f'{year_control_stroke_prop}%')
print('Proportion of strokes in 1-year treatment group:', f'{year_treatment_stroke_prop}%')

# difference in proportions (treatment - control)
year_stroke_prop_difference = year_treatment_stroke_prop - year_control_stroke_prop

print('Difference in stroke proportions:',\
    f'{abs(year_stroke_prop_difference)}%',\
    'more strokes in',\
    'treatment' if year_stroke_prop_difference >= 0 else 'control',
    'group'
)

Proportion of strokes in 1-year control group: 12.33%
Proportion of strokes in 1-year treatment group: 20.09%
Difference in stroke proportions: 7.76% more strokes in treatment group


**Do not generalize to all patients and all stents**: This study only considers one type of stent and the patients all had specific characteristics, and all volunteered to be part of the study.

## 1.1 Exercises

### 1.1.1

In [5]:
# data-only dataframe
migraine_df = pd.DataFrame(
    data= [
        [10, 33],
        [2, 44]
    ],
    index= [
        'Treatment',
        'Control'
    ],
    columns= [
        'Pain_free',
        'Not_pain_free'
    ]
)
# add totals row and column
migraine_df['Group_total'] = [sum(migraine_df.loc[idx]) for idx in migraine_df.index]
migraine_df.loc['Outcome_total'] = migraine_df.sum()

display(migraine_df)

# 1.1.1a
pct_treat_pf = migraine_df.loc['Treatment', 'Pain_free'] / migraine_df.loc['Treatment', 'Group_total']
print(f'Percent of pain-free patients in the treatment group: {round(pct_treat_pf*100, 2)}%')
# 1.1.1b
pct_cont_pf = migraine_df.loc['Control', 'Pain_free'] / migraine_df.loc['Control', 'Group_total']
print(f'Percent of pain-free patients in the control group: {round(pct_cont_pf*100, 2)}%')
# 1.1.1c
higher_pct_group = 'Treatment' if pct_treat_pf > pct_cont_pf else 'Control' 
print(f'The {higher_pct_group} group had a higher percentage of pain-free patients.')
# 1.1.1d
pct_difference = abs(pct_treat_pf - pct_cont_pf)
print(
    f'The difference of {round(pct_difference*100, 2)}% could be attributed to random chance,',
     'though this is unlikely.'
)

,Pain_free,Not_pain_free,Group_total
Treatment,10,33,43
Control,2,44,46
Outcome_total,12,77,89


Percent of pain-free patients in the treatment group: 23.26%
Percent of pain-free patients in the control group: 4.35%
The Treatment group had a higher percentage of pain-free patients.
The difference of 18.91% could be attributed to random chance, though this is unlikely.


### 1.1.2

In [6]:
# create full df (with totals)
sinusitis_df = pd.DataFrame(
    data= {
        'Improvement': [66, 65, 131],
        'No_improvement': [19, 16, 35],
        'Group_total': [85, 81, 166]
    },
    index= [
        'Treatment',
        'Control',
        'Outcome_total'
    ]
)

display(sinusitis_df)

#1.1.2a
pct_treat_i = sinusitis_df.loc['Treatment', 'Improvement'] / sinusitis_df.loc['Treatment', 'Group_total']
print(f'Percent of treatment patients with improvement in symptoms: {round(pct_treat_i*100, 2)}%')
#1.1.2b
pct_cont_i = sinusitis_df.loc['Control', 'Improvement'] / sinusitis_df.loc['Control', 'Group_total']
print(f'Percent of control patients with improvement in symptoms: {round(pct_cont_i*100, 2)}%')
#1.1.2c
higher_pct_group = 'Treatment' if pct_treat_i > pct_cont_i else 'Control'
print(f'The {higher_pct_group} group had a higher percentage of patients with improvment in symptoms.')
#1.1.2d
pct_difference = abs(pct_treat_i - pct_cont_i)
print(
    f'With a difference of {round(pct_difference*100, 2)}%,',
    'there does not seem to be a real difference between the two groups.'
)
print('The difference is likely due to random chance.')


,Improvement,No_improvement,Group_total
Treatment,66,19,85
Control,65,16,81
Outcome_total,131,35,166


Percent of treatment patients with improvement in symptoms: 77.65%
Percent of control patients with improvement in symptoms: 80.25%
The Control group had a higher percentage of patients with improvment in symptoms.
With a difference of 2.6%, there does not seem to be a real difference between the two groups.
The difference is likely due to random chance.


# Chapter 2: Summarizing Data

# Chapter 3: Probability

# Chapter 4: Distributions of Random Variables

In [7]:
# probability of finding first success in 
# Nth trial if
# P is probability of success and
# 1-P is probability of failure
def geometric_distribution(p:float, n:int):
    from math import sqrt

    probability = p * (1-p)**(n-1)
    mean = 1 / p
    variance = (1-p)/(p**2)
    std = math.sqrt(variance)

    return probability, mean, variance, std

In [8]:
# GP 4.23
print(round(1/0.7, 2))

1.43


In [9]:
prob = 0.7
p_list = []
for i in range(1, 4):
    p, mu, sigma, std = geometric_distribution(prob, i)
    p_list.append(p)
print(sum(p_list))
p_complement = (1-prob)**3
print(1-p_complement)

0.973
0.973


In [10]:
# 4.11a
prob = 0.25
mu = 1 / prob
std = math.sqrt((1-prob)/(prob**2))
print(prob, mu, std)

0.25 4.0 3.4641016151377544


In [11]:
#4.11b
prob = 1/6
mu = 1 / prob
std = math.sqrt((1-prob)/(prob**2))
print(prob, mu, std)

0.16666666666666666 6.0 5.477225575051661


In [12]:
#4.12a
with_replacement = (5/10)**2
without_replacement = (5/10) * (4/9)

print(with_replacement)
print(without_replacement)

0.25
0.2222222222222222


In [13]:
#4.12b
with_replacement = (5000/10000)**2
without_replacement = (5000/10000) * (4999/9999)

print(with_replacement)
print(without_replacement)

0.25
0.24997499749974997


In [14]:
#4.13a
blue_eyes = 0.125
brown_eyes = 0.75
green_eyes = 0.125

n_children = 3

prob = (1-blue_eyes)**(n_children-1) * blue_eyes
print(prob)

0.095703125


In [15]:
#4.13b
mu = 1 / blue_eyes
std = math.sqrt((1-blue_eyes)/(blue_eyes**2))
print(mu, std)

8.0 7.483314773547883


In [16]:
#4.14a
defect = 0.02
n_failure = 10

prob = (1-defect)**(n_failure-1) * defect
print(prob)

0.016674955242602995


In [17]:
#4.14b
prob = (1-defect)**100
print(prob)

0.13261955589475294


In [18]:
#4.14c
mu = 1 / defect
std = math.sqrt((1-defect)/(defect**2))
print(mu, std)

50.0 49.49747468305833


In [19]:
# 4.14d
defect = 0.05
mu = 1 / defect
std = math.sqrt((1-defect)/(defect**2))
print(mu, std)

20.0 19.493588689617926


In [20]:
# probability of finding k successes in n trials
def binomial_distribution(p:float, n:int, k:int):
    prob = math.comb(n, k) * (p**k) * ((1-p)**(n-k))
    mu = n * p
    var = n * p * (1-p)
    std = math.sqrt(var)

    return prob, mu, var, std

In [21]:
p = 0.7
n = 8
k = 5

results = [binomial_distribution(p, n, k)]

print(results[0])

(0.25412184000000004, 5.6, 1.6800000000000002, 1.2961481396815722)
